In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/My Drive/random_sample_astan.csv")

question = df.iloc[:, 1].tolist()
answer = df['پاسخ'].tolist()
answer
data = []
x = ""

for i in range(len(question)):
  # x += answer[i]
  x += str(question[i])
  data.append(x)
  x = ""


In [ ]:
#  seprate 80% of data for train
train_data = data[:40000]

#  seprate 20% data for test
test_data = data[40000:46579]


In [ ]:
#  stop words
SW = []
 
with open('/content/drive/My Drive/verbal') as file :
  for line in file :
    SW.append(line)

with open('/content/drive/My Drive/short') as file :
  for line in file :
    SW.append(line)

with open('/content/drive/My Drive/persian') as file :
  for line in file :
    SW.append(line)

with open('/content/drive/My Drive/nonverbal') as file :
  for line in file :
    SW.append(line)

SW = [ s.replace('\n' , '') for s in SW]
SW = list(set(SW))



In [ ]:
#####################
#  BOW
#####################

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans

CountVec = CountVectorizer(stop_words= SW)
vec = CountVec.fit(train_data)
vectorized = vec.transform(train_data)

number_of_clusters= 3
kmeans = KMeans(n_clusters=number_of_clusters, max_iter = 50)
kmeans.fit(vectorized)

print("Top terms per cluster:")
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vec.get_feature_names()

for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :10]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))



In [ ]:
from sklearn.metrics import silhouette_score
score = silhouette_score(vectorized, kmeans.labels_, metric= 'euclidean')
print(" best score of  BOW classification train data :")
score


 best score of  BOW classification train data :


0.03289788133115093

In [ ]:
results_train = pd.DataFrame({
    'text': train_data[:20],
    'category': kmeans.labels_[:20]
})

print("20 first data in train data with cluster number:")

list_train = results_train.values.tolist()

for i in range(len(list_train)):
  print(data[i], list_train[i][1])

In [ ]:
vec_test = vec.transform(test_data)
kmeans.predict(vec_test)

print("20 first data in test data with cluster number:")
results_test = pd.DataFrame({
    'text': test_data[:20],
    'category': kmeans.labels_[:20]
})

list_test = results_test.values.tolist()

for i in range(len(list_test)):
  print(data[i+40000], list_test[i][1])

In [ ]:
#####################
#  tfidf
#####################
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

vectorizer = TfidfVectorizer(stop_words= SW)
vec = vectorizer.fit(train_data)   
vectorized = vec.transform(train_data) 

number_of_clusters= 20
km = KMeans(n_clusters=number_of_clusters)
# Normally people fit the matrix
km.fit(vectorized)

print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vec.get_feature_names()

for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :10]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))



In [ ]:
from sklearn.metrics import silhouette_score
score = silhouette_score(vectorized, km.labels_, metric= 'euclidean')
print(" best score of classification tfidf:")
score

 best score of classification tfidf:


0.007931652095494445

In [ ]:
results_train = pd.DataFrame({
    'text': train_data[:20],
    'category': km.labels_[:20]
})

print("20 first data in train data with cluster number:")

list_train = results_train.values.tolist()

for i in range(len(list_train)):
  print(data[i], list_train[i][1])


In [ ]:
print("20 first data in test data with cluster number:")

list2Vec = vec.transform(test_data)
km.predict(list2Vec)

results_test = pd.DataFrame({
    'text': test_data[:20],
    'category': km.labels_[:20]
})

list_test = results_test.values.tolist()

for i in range(len(list_test)):
  print(data[i+40000], list_test[i][1])

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git

In [ ]:
!cd fastText
!pip install fastText

In [ ]:
#####################
#  fasttext
#####################

fasttext.util.download_model('fa', if_exists='ignore')  # English


In [ ]:
import fasttext.util
import fasttext
ft = fasttext.load_model('cc.fa.300.bin')


In [ ]:
list_vect = []
for i in range(len(train_data)):
  vect = ft.get_sentence_vector(train_data[i]) # 1 sentence
  list_vect.append(vect)

In [ ]:
from sklearn.cluster import KMeans
number_of_clusters= 3
km = KMeans(n_clusters=number_of_clusters)
km.fit(list_vect)


In [ ]:
from sklearn.metrics import silhouette_score
score = silhouette_score(list_vect, km.labels_, metric= 'euclidean')
print("best score of classification fast text for train data:")
score



In [ ]:
list_vect_test = []
for i in range(len(test_data)):
  vect = ft.get_sentence_vector(test_data[i]) 
  list_vect_test.append(vect)

In [ ]:
km.predict(list_vect_test)
